[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pdf-tools/components-code-sample-hub/blob/main/jupyter/pdftools_sdk/pdftools_sdk_built_in_certify.ipynb)

In [ ]:
%pip install https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/productkits/PDFSDK/latest/pdftools_sdk-latest.tar.gz
%pip install ipython

# Certify a PDF
This type of signature allows the PDF author to specify
which types of modifications are permissible after
signing.
These signatures are also known as Modification Detection
and Prevention (MDP) signatures.

The signing certificate is read from a password-protected
PKCS#12 file (.pfx or .p12).


In [ ]:
import io
from pdftools_sdk.pdf import Document
from pdftools_sdk.sign import Signer
from pdftools_sdk.crypto.providers.built_in import Provider
from pdftools_sdk.crypto import ValidationInformation

In [ ]:
# Download a file from a given URL and save it to the local system
def prepare_file(url: str, path: str):
    import requests
    response = requests.get(url)
    response.raise_for_status()

    with open(path, 'wb') as f:
        f.write(response.content)

In [ ]:
# Set input arguments
certificate_file = 'INSERT-PFX-FILE'  # Placeholder for PFX file
password = 'INSERT-PASSWORD'
input_url = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/InvoiceNone.pdf'
input_path = 'InvoiceNone.pdf'
prepare_file(input_url, input_path)
output_path = 'Certified.pdf'

In [ ]:
def certify_document(certificate_file: str, password: str, input_path: str, output_path: str):
    # Create a session to the built-in cryptographic provider
    with Provider() as session:
        with io.FileIO(certificate_file, 'rb') as pfx_stream:
            # Create signature configuration from PFX (or P12) file
            signature = session.create_signature_from_certificate(pfx_stream, password)

            # Embed validation information to enable the long-term validation (LTV) of the signature
            signature.validation_information = ValidationInformation.EMBED_IN_DOCUMENT

            # Open input document
            with io.FileIO(input_path, 'rb') as in_stream:
                with Document.open(in_stream) as input_document:

                    # Create stream for output file
                    with io.FileIO(output_path, 'wb+') as output_stream:
                        # Certify the document with the MDP signature
                        signer = Signer()
                        signer.certify(input_document, signature, output_stream)

In [ ]:
try:
    # By default, a test license key is active. In this case, a watermark is added to the output. 
    # If you have a license key, please uncomment the following call and set the license key.
    # from pdftools_sdk.sdk import Sdk
    # Sdk.initialize("INSERT-LICENSE-KEY")

    # Certify a PDF document
    certify_document(certificate_file, password, input_path, output_path)

    print(f"Successfully created file {output_path}")
except Exception as e:
    print(f"An error occurred: {e}")